# Python+Z3 Contraint Programming Approach to S*FSM Encodings
* Author: Mike Borowczak
* Orignal Date: Circa 2012
* Updated : 04/2020
* Major Changes:
  * Python 3.x+ 
  * Clean up of Log2 Use
  * Clean up of Hamming Functions
* Work based on the following publications:
  * M. Borowczak and R. Vemuri, "S*FSM: A Paradigm Shift for Attack Resistant FSM Designs and Encodings," 2012 ASE/IEEE International Conference on BioMedical Computing (BioMedCom), Washington, DC, 2012, pp. 96-100, doi: 10.1109/BioMedCom.2012.22. See: https://ieeexplore.ieee.org/document/6516435
  * M. Borowczak and R. Vemuri, "Enabling Side Channel Secure FSMs in the Presence of Low Power Requirements," 2014 IEEE Computer Society Annual Symposium on VLSI, Tampa, FL, 2014, pp. 232-235, doi: 10.1109/ISVLSI.2014.78. See: https://ieeexplore.ieee.org/document/6903365
  * Borowczak, Mike; Vemuri, Ranga: 'Mitigating information leakage during critical communication using S*FSM', IET Computers &amp; Digital Techniques, 2019, 13, (4), p. 292-301, DOI: 10.1049/iet-cdt.2018.5186 See https://digital-library.theiet.org/content/journals/10.1049/iet-cdt.2018.5186

In [2]:
from z3 import *
from math import *

## Helper Functions 

### Max Bits Based on Value

In [3]:
def IntCeilLog2(val): return int(ceil(log2(val)))

### Efficent Hamming Computations
- Use Bit Vect extraction of ith bit; and sum+store in a bitvector of size log_2(bv_length)

In [4]:
def HW(bv):
    return Sum([
        ZeroExt(IntCeilLog2(bv.size()), Extract(i,i,bv)) 
        for i in range(bv.size())
    ])

In [5]:
def HD(bv1,bv2):
    return Sum([
        ZeroExt(IntCeilLog2(bv1.size()), Extract(i,i, bv1 ^ bv2)) 
        for i in range(bv1.size())
    ])

## Contraint Approach to Encoding

### Unique Encoding 
Use of `Distinct` to guarentee states do not share an encoding

In [6]:
for bits in range(6,  IntCeilLog2(6)-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)

    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    
    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()
        

Constraints Satisified In 6 Bits
	state_st2	->	000011
	state_snt	->	000101
	state_wt	->	111001
	state_snt2	->	111010
	state_wnt	->	011001
	state_st	->	010110
Constraints Satisified In 5 Bits
	state_snt2	->	00011
	state_wnt	->	01000
	state_st	->	11110
	state_st2	->	01100
	state_snt	->	00001
	state_wt	->	10110
Constraints Satisified In 4 Bits
	state_st2	->	0100
	state_snt	->	0011
	state_wt	->	1101
	state_snt2	->	0010
	state_wnt	->	0110
	state_st	->	1100
Constraints Satisified In 3 Bits
	state_snt2	->	110
	state_wnt	->	000
	state_st	->	111
	state_st2	->	001
	state_snt	->	100
	state_wt	->	011


### Enabling a constant hamming weight across all states


In [7]:
for bits in range(6,  IntCeilLog2(6)-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)

    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    #Hamming Weight Constraints (#s-1)
    s.add(HW(state_st) == HW(state_st2))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wnt))
    s.add(HW(state_st) == HW(state_snt))    
    s.add(HW(state_st) == HW(state_snt2))
        
    #Hamming Distance Constraints (#s-1)
    s.add(HD(state_st2,state_st)==HD(state_st2,state_wt))
    s.add(HD(state_st,state_st2)==HD(state_st,state_wt))    
    s.add(HD(state_wt,state_st)==HD(state_wt,state_wnt))
    s.add(HD(state_snt2,state_snt)==HD(state_snt2,state_wnt))
    s.add(HD(state_snt,state_snt2)==HD(state_snt,state_wnt))    
    s.add(HD(state_wnt,state_snt)==HD(state_wnt,state_wt))
    
    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()
        

Constraints Satisified In 6 Bits
	state_st2	->	011110
	state_snt	->	110110
	state_wt	->	110101
	state_snt2	->	101101
	state_wnt	->	011011
	state_st	->	101011
Constraints Satisified In 5 Bits
	state_snt2	->	10110
	state_wnt	->	11010
	state_st	->	01101
	state_st2	->	11001
	state_snt	->	11100
	state_wt	->	01011
Constraints Satisified In 4 Bits
	state_st2	->	0011
	state_snt	->	0101
	state_wt	->	1010
	state_snt2	->	1100
	state_wnt	->	1001
	state_st	->	0110
Constraints Not Satisified in 3 Bits


### Enabling a constant hamming weight and distance across all states

In [8]:
for bits in range(10,  int(log2(6))-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)
    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    s.add(HW(state_st) == HW(state_st2))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wnt))
    s.add(HW(state_st) == HW(state_snt))    
    s.add(HW(state_st) == HW(state_snt2))
    
    s.add(HD(state_st2,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st)==HD(state_st,state_st2))

    s.add(HD(state_st,state_st2)==HD(state_st,state_st2))

    s.add(HD(state_wt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st2)==HD(state_st,state_st2))


    s.add(HD(state_st,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_wnt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_snt,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt)==HD(state_st,state_st2))

    s.add(HD(state_snt2,state_snt)==HD(state_st,state_st2))

    
    s.add(HD(state_st,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_snt2)==HD(state_st,state_st2))


    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()

Constraints Satisified In 10 Bits
	state_snt2	->	1010111111
	state_wnt	->	1011111011
	state_st	->	1001111111
	state_st2	->	1011110111
	state_snt	->	0011111111
	state_wt	->	1011101111
Constraints Satisified In 9 Bits
	state_snt2	->	111110001
	state_wnt	->	110111010
	state_st	->	010110111
	state_st2	->	100111101
	state_snt	->	011111100
	state_wt	->	001111011
Constraints Satisified In 8 Bits
	state_st2	->	10110111
	state_snt	->	11110011
	state_wt	->	11100111
	state_snt2	->	11110101
	state_wnt	->	11110110
	state_st	->	11010111
Constraints Satisified In 7 Bits
	state_snt2	->	1001110
	state_wnt	->	1100011
	state_st	->	0101101
	state_st2	->	1110100
	state_snt	->	1011001
	state_wt	->	0111010
Constraints Satisified In 6 Bits
	state_st2	->	101111
	state_snt	->	111110
	state_wt	->	111011
	state_snt2	->	110111
	state_wnt	->	011111
	state_st	->	111101
Constraints Not Satisified in 5 Bits
Constraints Not Satisified in 4 Bits
Constraints Not Satisified in 3 Bits
Constraints Not Satisified in 2 Bits


### Enabling a constant hamming weight and specific distance across all states

In [9]:
for bits in range(10,  int(log2(6))-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)
    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    s.add(HW(state_st) == HW(state_st2))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wnt))
    s.add(HW(state_st) == HW(state_snt))    
    s.add(HW(state_st) == HW(state_snt2))
    
    s.add(HD(state_st2,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st)==HD(state_st,state_st2))

    s.add(HD(state_st,state_st2)==HD(state_st,state_st2))

    s.add(HD(state_wt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st2)==HD(state_st,state_st2))


    s.add(HD(state_st,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_wnt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_snt,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt)==HD(state_st,state_st2))
    
    s.add(HD(state_snt2,state_snt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_snt2)==HD(state_st,state_st2))
    
    s.add(HD(state_snt,state_snt2)==2)

    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()

Constraints Satisified In 10 Bits
	state_snt2	->	0100001000
	state_wnt	->	0100100000
	state_st	->	0100000010
	state_st2	->	0101000000
	state_snt	->	0110000000
	state_wt	->	1100000000
Constraints Satisified In 9 Bits
	state_snt2	->	111011111
	state_wnt	->	111110111
	state_st	->	111111110
	state_st2	->	011111111
	state_snt	->	110111111
	state_wt	->	111111101
Constraints Satisified In 8 Bits
	state_st2	->	10000101
	state_snt	->	10100100
	state_wt	->	10001100
	state_snt2	->	11000100
	state_wnt	->	10010100
	state_st	->	10000110
Constraints Satisified In 7 Bits
	state_snt2	->	1111110
	state_wnt	->	1110111
	state_st	->	1011111
	state_st2	->	1111011
	state_snt	->	1111101
	state_wt	->	0111111
Constraints Satisified In 6 Bits
	state_st2	->	100000
	state_snt	->	000100
	state_wt	->	000010
	state_snt2	->	000001
	state_wnt	->	010000
	state_st	->	001000
Constraints Not Satisified in 5 Bits
Constraints Not Satisified in 4 Bits
Constraints Not Satisified in 3 Bits
Constraints Not Satisified in 2 Bits


### Enabling a specific constant hamming weight and specific distance across all states

In [10]:
for bits in range(10,  int(log2(6))-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)
    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    s.add(HW(state_st) == HW(state_st2))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wnt))
    s.add(HW(state_st) == HW(state_snt))    
    s.add(HW(state_st) == HW(state_snt2))
    
    s.add(HD(state_st2,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st)==HD(state_st,state_st2))

    s.add(HD(state_st,state_st2)==HD(state_st,state_st2))

    s.add(HD(state_wt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st2)==HD(state_st,state_st2))


    s.add(HD(state_st,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_wnt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_snt,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt)==HD(state_st,state_st2))
    
    s.add(HD(state_snt2,state_snt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_snt2)==HD(state_st,state_st2))
    
    s.add(HD(state_snt,state_snt2)==2)
    s.add(HW(state_snt)==1)

    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()

Constraints Satisified In 10 Bits
	state_snt2	->	0000000001
	state_wnt	->	0000100000
	state_st	->	0001000000
	state_st2	->	0000010000
	state_snt	->	0010000000
	state_wt	->	0000001000
Constraints Satisified In 9 Bits
	state_snt2	->	000000100
	state_wnt	->	000010000
	state_st	->	010000000
	state_st2	->	001000000
	state_snt	->	000100000
	state_wt	->	000000010
Constraints Satisified In 8 Bits
	state_st2	->	10000000
	state_snt	->	00010000
	state_wt	->	00100000
	state_snt2	->	01000000
	state_wnt	->	00000001
	state_st	->	00000010
Constraints Satisified In 7 Bits
	state_snt2	->	0001000
	state_wnt	->	0010000
	state_st	->	0000010
	state_st2	->	0000001
	state_snt	->	0100000
	state_wt	->	0000100
Constraints Satisified In 6 Bits
	state_st2	->	000010
	state_snt	->	010000
	state_wt	->	001000
	state_snt2	->	000100
	state_wnt	->	000001
	state_st	->	100000
Constraints Not Satisified in 5 Bits
Constraints Not Satisified in 4 Bits
Constraints Not Satisified in 3 Bits
Constraints Not Satisified in 2 Bits


### Enabling a specific constant hamming weight = specific distance across all states

In [11]:
for bits in range(10,  int(log2(6))-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)
    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    s.add(HW(state_st) == HW(state_st2))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wnt))
    s.add(HW(state_st) == HW(state_snt))    
    s.add(HW(state_st) == HW(state_snt2))
    
    s.add(HD(state_st2,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st)==HD(state_st,state_st2))

    s.add(HD(state_st,state_st2)==HD(state_st,state_st2))

    s.add(HD(state_wt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st2)==HD(state_st,state_st2))


    s.add(HD(state_st,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_wnt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_snt,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt)==HD(state_st,state_st2))
    
    s.add(HD(state_snt2,state_snt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_snt2)==HD(state_st,state_st2))
    
    s.add(HD(state_snt,state_snt2)==2)
    s.add(HW(state_snt)==HD(state_snt,state_snt2))

    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()

Constraints Satisified In 10 Bits
	state_snt2	->	1010000000
	state_wnt	->	0011000000
	state_st	->	0010000010
	state_st2	->	0010010000
	state_snt	->	0110000000
	state_wt	->	0010001000
Constraints Satisified In 9 Bits
	state_snt2	->	000011000
	state_wnt	->	010010000
	state_st	->	000010010
	state_st2	->	000110000
	state_snt	->	100010000
	state_wt	->	001010000
Constraints Satisified In 8 Bits
	state_st2	->	10000010
	state_snt	->	01000010
	state_wt	->	00001010
	state_snt2	->	00000110
	state_wnt	->	00000011
	state_st	->	00010010
Constraints Satisified In 7 Bits
	state_snt2	->	1010000
	state_wnt	->	0010100
	state_st	->	0010010
	state_st2	->	0011000
	state_snt	->	0110000
	state_wt	->	0010001
Constraints Not Satisified in 6 Bits
Constraints Not Satisified in 5 Bits
Constraints Not Satisified in 4 Bits
Constraints Not Satisified in 3 Bits
Constraints Not Satisified in 2 Bits


### Enabling constant hamming weight = distance across all states

In [12]:
for bits in range(10,  int(log2(6))-1, -1):
    state_st, state_st2, state_wt, state_wnt, state_snt, state_snt2 =\
    BitVecs('state_st state_st2 state_wt state_wnt state_snt state_snt2',bits)
    s = Solver();
    s.set("timeout", 1000000)
    s.add(Distinct(state_st,state_st2,state_wt,state_wnt,state_snt,state_snt2))
    s.add(HW(state_st) == HW(state_st2))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wt))
    s.add(HW(state_st) == HW(state_wnt))
    s.add(HW(state_st) == HW(state_snt))    
    s.add(HW(state_st) == HW(state_snt2))
    
    s.add(HD(state_st2,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st)==HD(state_st,state_st2))

    s.add(HD(state_st,state_st2)==HD(state_st,state_st2))

    s.add(HD(state_wt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_st2)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_st2)==HD(state_st,state_st2))


    s.add(HD(state_st,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_wnt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_wt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_snt,state_wnt)==HD(state_st,state_st2))
    s.add(HD(state_snt2,state_wnt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt)==HD(state_st,state_st2))
    
    s.add(HD(state_snt2,state_snt)==HD(state_st,state_st2))

    s.add(HD(state_st,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_st2,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_wnt,state_snt2)==HD(state_st,state_st2))
    s.add(HD(state_snt,state_snt2)==HD(state_st,state_st2))
    
    s.add(HW(state_snt)==HD(state_snt,state_snt2))

    if(s.check() == sat):
        print("Constraints Satisified In %d Bits" %(bits)),
        m = s.model()
        for d in m.decls():
            print('\t{state}\t->\t{encode:0{fieldsize}b}'.format(state=d.name(),encode=m[d].as_long(),fieldsize=bits))
    else:
        print("Constraints Not Satisified in %d Bits" %(bits))
        sys.stdout.flush()

Constraints Satisified In 10 Bits
	state_snt2	->	0000001111
	state_wnt	->	0100101100
	state_st	->	0000110110
	state_st2	->	1000101010
	state_snt	->	0000111001
	state_wt	->	0100100011
Constraints Satisified In 9 Bits
	state_snt2	->	000001111
	state_wnt	->	100011010
	state_st	->	010011100
	state_st2	->	100101100
	state_snt	->	000110110
	state_wt	->	110000110
Constraints Satisified In 8 Bits
	state_st2	->	01101001
	state_snt	->	10101100
	state_wt	->	11001010
	state_snt2	->	11110000
	state_wnt	->	01100110
	state_st	->	01011100
Constraints Satisified In 7 Bits
	state_snt2	->	1101001
	state_wnt	->	1110010
	state_st	->	1010101
	state_st2	->	0011011
	state_snt	->	0111100
	state_wt	->	0100111
Constraints Not Satisified in 6 Bits
Constraints Not Satisified in 5 Bits
Constraints Not Satisified in 4 Bits
Constraints Not Satisified in 3 Bits
Constraints Not Satisified in 2 Bits
